In [ ]:
# ======================================================
# 설치
# ======================================================
!pip -q install konlpy mecab-python3 emoji pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.8/588.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 7.5 MB/s eta 0:00:00
[알림] Mecab 사용 불가 → Okt로 진행합니다.
사용 형태소 분석기: okt


,raw,normalized,base,k_ㅋㅋ,h_ㅎㅎ,tt_ㅠ,ts_ㅜ,boost,total,label
0,와 ㅋㅋㅋㅋ 오늘 공연 미쳤다 🔥🔥🔥,와 ㅋㅋ 오늘 공연 미쳤다 EMOJI EMOJI EMOJI,0,4,0,0,0,0.805,0.805,neutral
1,비가 와서 집콕... ㅠㅠ 커피라도 마셔야지 ☕️,비가 와서 집콕... ㅠㅠ 커피라도 마셔야지 EMOJI,0,0,0,2,0,-0.769,-0.769,neutral
2,헐ㅋㅋ 어제 경기 ㄹㅇ 레전드;; 대박이다!!!,헐ㅋㅋ 어제 경기 ㄹㅇ 레전드;; 대박이다!,1,2,0,0,0,0.549,1.549,positive
3,하하하 이것도 웃기네 ㅎㅎㅎ 😂😂,하하하 이것도 웃기네 ㅎㅎ EMOJI EMOJI,0,0,3,0,0,0.693,0.693,neutral
4,콘서트 취소라니 최악... ㅠㅠㅠㅠ,콘서트 취소라니 최악... ㅠㅠ,-1,0,0,4,0,-1.127,-2.127,negative
5,노잼이었지만 친구랑 있어서 그나마 괜찮았다,노잼이었지만 친구랑 있어서 그나마 괜찮았다,-1,0,0,0,0,0.000,-1.000,negative
6,별로였는데 ㅋㅋ 그래도 사람 많아서 신났다,별로였는데 ㅋㅋ 그래도 사람 많아서 신났다,-1,2,0,0,0,0.549,-0.451,neutral


In [1]:

import re, emoji
import numpy as np
import pandas as pd
from konlpy.tag import Okt

# 형태소 분석기 (Mecab → Okt 폴백)
tokenizer_name="okt"; okt=Okt(); mecab=None
try:
    from konlpy.tag import Mecab
    mecab=Mecab(); tokenizer_name="mecab"
except:
    print("[알림] Mecab 사용 불가 → Okt로 진행합니다.")
print("사용 형태소 분석기:", tokenizer_name)

def morphs(text, stem=True):
    if tokenizer_name=="mecab":
        # MeCab은 표제어화가 기본적으로 없으니 stem 파라미터 무시
        return [t for t in mecab.morphs(text) if len(t)>=2]
    else:
        return [t for t in okt.morphs(text, stem=stem) if len(t)>=2]

# ======================================================
# 1) 정규화 + 강도(횟수) 추출
# ======================================================
PATTERNS = {
    r"(ㅋ)\1{1,}": "ㅋㅋ",   # ㅋ 반복 → ㅋㅋ
    r"(ㅎ)\1{1,}": "ㅎㅎ",   # ㅎ 반복 → ㅎㅎ
    r"(ㅠ)\1{1,}": "ㅠㅠ",   # ㅠ 반복 → ㅠㅠ
    r"(ㅜ)\1{1,}": "ㅜㅜ",   # ㅜ 반복 → ㅜㅜ
    r"(!)\1{1,}": "!",
    r"([?])\1{1,}": "?",
}

def count_intensity(raw: str):
    """
    ㅋ/ㅎ/ㅠ/ㅜ 반복 묶음의 '길이'를 모두 합산.
    예) 'ㅋㅋㅋㅋ' → 4, 'ㅎㅎㅎ' → 3, 'ㅠㅠㅠㅠ' → 4, 그룹이 여러 개면 전부 더함.
    """
    k_len = sum(len(m.group(0)) for m in re.finditer(r"(ㅋ)\1*", raw))
    h_len = sum(len(m.group(0)) for m in re.finditer(r"(ㅎ)\1*", raw))
    u_len = sum(len(m.group(0)) for m in re.finditer(r"(ㅠ)\1*", raw))
    j_len = sum(len(m.group(0)) for m in re.finditer(r"(ㅜ)\1*", raw))
    return {"k": k_len, "h": h_len, "tt": u_len, "ts": j_len}  # tt=ㅠ, ts=ㅜ

def normalize_text(raw: str):
    s = raw
    for pat, rep in PATTERNS.items():
        s = re.sub(pat, rep, s)
    s = emoji.replace_emoji(s, replace=" EMOJI ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ======================================================
# 2) 베이스 감성(간단 사전) + 강도 가중 결합
# ======================================================
POS_LEX = set("""
좋다 최고 재밌다 재밌 재미 감동 미쳤다 멋지다 예쁘다 대박 명작 추천 만족 행복 웃기다 신난다 설렌다 흥미롭다
""".split())
NEG_LEX = set("""
별로 최악 구리다 싫다 지루하다 짜증 화남 분노 실망 아쉽다 우울 슬프다 노잼 망작 최악이다 불편 불만
""".split())

def base_sentiment_score(text: str):
    """아주 단순한 사전 매칭 점수: +1(긍정어), -1(부정어) 합산"""
    toks = morphs(text, stem=True)
    score = 0
    for t in toks:
        if t in POS_LEX: score += 1
        if t in NEG_LEX: score -= 1
    return score

def combined_sentiment(raw: str,
                       w_k=0.5, w_h=0.5, w_tt=-0.7, w_ts=-0.7,
                       use_log=True):
    """
    최종 점수 = 베이스 감성 + 강도 가중치
      - ㅋㅋ/ㅎㅎ: 긍정 강화(+)
      - ㅠㅠ/ㅜㅜ: 부정 강화(-)
      - use_log=True면 log1p로 완만하게(폭주 방지)
    """
    intens = count_intensity(raw)
    normed = normalize_text(raw)
    base = base_sentiment_score(normed)

    if use_log:
        k = np.log1p(intens["k"])
        h = np.log1p(intens["h"])
        tt = np.log1p(intens["tt"])
        ts = np.log1p(intens["ts"])
    else:
        k, h, tt, ts = intens["k"], intens["h"], intens["tt"], intens["ts"]

    boost = w_k*k + w_h*h + w_tt*tt + w_ts*ts
    total = base + boost

    # 라벨링(간단 임계)
    label = "neutral"
    if total >= 1.0: label = "positive"
    if total <= -1.0: label = "negative"

    return {
        "raw": raw,
        "normalized": normed,
        "base": base,
        "k_ㅋㅋ": intens["k"],
        "h_ㅎㅎ": intens["h"],
        "tt_ㅠ": intens["tt"],
        "ts_ㅜ": intens["ts"],
        "boost": round(float(boost), 3),
        "total": round(float(total), 3),
        "label": label
    }

# ======================================================
# 3) 데모: 샘플 문장 평가
# ======================================================
SENTS = [
    "와 ㅋㅋㅋㅋ 오늘 공연 미쳤다 🔥🔥🔥",
    "비가 와서 집콕... ㅠㅠ 커피라도 마셔야지 ☕️",
    "헐ㅋㅋ 어제 경기 ㄹㅇ 레전드;; 대박이다!!!",
    "하하하 이것도 웃기네 ㅎㅎㅎ 😂😂",
    "콘서트 취소라니 최악... ㅠㅠㅠㅠ",
    "노잼이었지만 친구랑 있어서 그나마 괜찮았다",
    "별로였는데 ㅋㅋ 그래도 사람 많아서 신났다",
]

rows = [combined_sentiment(s) for s in SENTS]
df = pd.DataFrame(rows, columns=[
    "raw","normalized","base","k_ㅋㅋ","h_ㅎㅎ","tt_ㅠ","ts_ㅜ","boost","total","label"
])
df


사용 형태소 분석기: mecab


,raw,normalized,base,k_ㅋㅋ,h_ㅎㅎ,tt_ㅠ,ts_ㅜ,boost,total,label
0,와 ㅋㅋㅋㅋ 오늘 공연 미쳤다 🔥🔥🔥,와 ㅋㅋ 오늘 공연 미쳤다 EMOJI EMOJI EMOJI,0,4,0,0,0,0.805,0.805,neutral
1,비가 와서 집콕... ㅠㅠ 커피라도 마셔야지 ☕️,비가 와서 집콕... ㅠㅠ 커피라도 마셔야지 EMOJI,0,0,0,2,0,-0.769,-0.769,neutral
2,헐ㅋㅋ 어제 경기 ㄹㅇ 레전드;; 대박이다!!!,헐ㅋㅋ 어제 경기 ㄹㅇ 레전드;; 대박이다!,1,2,0,0,0,0.549,1.549,positive
3,하하하 이것도 웃기네 ㅎㅎㅎ 😂😂,하하하 이것도 웃기네 ㅎㅎ EMOJI EMOJI,0,0,3,0,0,0.693,0.693,neutral
4,콘서트 취소라니 최악... ㅠㅠㅠㅠ,콘서트 취소라니 최악... ㅠㅠ,-1,0,0,4,0,-1.127,-2.127,negative
5,노잼이었지만 친구랑 있어서 그나마 괜찮았다,노잼이었지만 친구랑 있어서 그나마 괜찮았다,0,0,0,0,0,0.000,0.000,neutral
6,별로였는데 ㅋㅋ 그래도 사람 많아서 신났다,별로였는데 ㅋㅋ 그래도 사람 많아서 신났다,-1,2,0,0,0,0.549,-0.451,neutral


# 도메인, 이모지 추가


In [3]:
EMOJI_WEIGHT = {
    "😂": +0.9, "😆": +0.8, "🤣": +1.0, "😊": +0.6, "😄": +0.7, "😍": +0.9, "🤩": +0.9, "😅": +0.5,
    "😭": -1.0, "😢": -0.8, "😡": -0.9, "🤬": -1.0, "😔": -0.7, "😞": -0.7, "🤢": -0.8, "😐": -0.1
}

In [15]:
def count_emoji_intensity(text: str):
    """
    EMOJI_WEIGHT에 있는 이모지들의 강도를 계산하는 함수
    각 이모지의 가중치 * 개수를 합산하여 반환
    """
    total_score = 0
    emoji_counts = {}
    
    # 텍스트에서 모든 이모지 추출
    for char in text:
        if emoji.is_emoji(char) and char in EMOJI_WEIGHT:
            emoji_counts[char] = emoji_counts.get(char, 0) + 1
    
    # 각 이모지의 가중치 * 개수 계산
    for emo, count in emoji_counts.items():
        count = np.log1p(count)
        total_score += EMOJI_WEIGHT[emo] * count
    
    return {
        "emoji_score": round(total_score, 3),
        "emoji_counts": emoji_counts
    }

def combined_sentiment_with_emoji(raw: str,
                                 w_k=0.5, w_h=0.5, w_tt=-0.7, w_ts=-0.7, w_emoji=0.6,
                                 use_log=True):
    """
    최종 점수 = 베이스 감성 + 강도 가중치 + 이모지 가중치
      - ㅋㅋ/ㅎㅎ: 긍정 강화(+)
      - ㅠㅠ/ㅜㅜ: 부정 강화(-)
      - 이모지: EMOJI_WEIGHT 기반 감정 점수
    """
    intens = count_intensity(raw)
    emoji_result = count_emoji_intensity(raw)
    normed = normalize_text(raw)
    base = base_sentiment_score(normed)

    if use_log:
        k = np.log1p(intens["k"])
        h = np.log1p(intens["h"])
        tt = np.log1p(intens["tt"])
        ts = np.log1p(intens["ts"])
    else:
        k, h, tt, ts = intens["k"], intens["h"], intens["tt"], intens["ts"]

    boost = w_k*k + w_h*h + w_tt*tt + w_ts*ts
    emoji_boost = w_emoji * emoji_result["emoji_score"]
    total = base + boost + emoji_boost

    # 라벨링(간단 임계)
    label = "neutral"
    if total >= 1.0: label = "positive"
    if total <= -1.0: label = "negative"

    return {
        "raw": raw,
        "normalized": normed,
        "base": base,
        "k_ㅋㅋ": intens["k"],
        "h_ㅎㅎ": intens["h"],
        "tt_ㅠ": intens["tt"],
        "ts_ㅜ": intens["ts"],
        "emoji_score": emoji_result["emoji_score"],
        "emoji_counts": emoji_result["emoji_counts"],
        "boost": round(float(boost), 3),
        "emoji_boost": round(float(emoji_boost), 3),
        "total": round(float(total), 3),
        "label": label
    }

# 테스트
test_sentences = [
    "레전드 공연... 무대미쳤다 😂😂 강추",
    "가격창렬에 음향구림.. 환불각 😡",
    "노잼이었는데 ㅠㅠ 배우 연기는 좋았다",
    "대박 갓작!! 재관람 의사 100% 🤩🤩",
]

for sent in test_sentences:
    result = combined_sentiment_with_emoji(sent)
    print(f"문장: {sent}")
    print(f"총점: {result['total']}, 라벨: {result['label']}")
    print(f"이모지점수: {result['emoji_score']}, 개수: {result['emoji_counts']}")
    print("-" * 50)

문장: 레전드 공연... 무대미쳤다 😂😂 강추
총점: 0.593, 라벨: neutral
이모지점수: 0.989, 개수: {'😂': 2}
--------------------------------------------------
문장: 가격창렬에 음향구림.. 환불각 😡
총점: -0.374, 라벨: neutral
이모지점수: -0.624, 개수: {'😡': 1}
--------------------------------------------------
문장: 노잼이었는데 ㅠㅠ 배우 연기는 좋았다
총점: -0.769, 라벨: neutral
이모지점수: 0, 개수: {}
--------------------------------------------------
문장: 대박 갓작!! 재관람 의사 100% 🤩🤩
총점: 1.593, 라벨: positive
이모지점수: 0.989, 개수: {'🤩': 2}
--------------------------------------------------


In [16]:
# =========================
# 추가 데모
# =========================
SENTS = [
    "레전드 공연... 무대미쳤다 😂😂 강추",
    "가격창렬에 음향구림.. 환불각 😡",
    "노잼이었는데 ㅠㅠ 배우 연기는 좋았다",
    "대박 갓작!! 재관람 의사 100% 🤩🤩",
    "좌석별로 시야가 너무 별로였음 😞",
    "그럭저럭이었지만 ㅎㅎ 분위기는 좋았음",
    "콘서트 진행엉망.. 최악 😭😭",
]

In [17]:
# ======================================================
# 3) 데모: 샘플 문장 평가
# ======================================================


rows = [combined_sentiment_with_emoji(s) for s in SENTS]
df = pd.DataFrame(rows, columns=[
    "raw","normalized","base","k_ㅋㅋ","h_ㅎㅎ","tt_ㅠ","ts_ㅜ","emoji_score","boost","total","label"
])
df


,raw,normalized,base,k_ㅋㅋ,h_ㅎㅎ,tt_ㅠ,ts_ㅜ,emoji_score,boost,total,label
0,레전드 공연... 무대미쳤다 😂😂 강추,레전드 공연... 무대미쳤다 EMOJI EMOJI 강추,0,0,0,0,0,0.989,0.000,0.593,neutral
1,가격창렬에 음향구림.. 환불각 😡,가격창렬에 음향구림.. 환불각 EMOJI,0,0,0,0,0,-0.624,0.000,-0.374,neutral
2,노잼이었는데 ㅠㅠ 배우 연기는 좋았다,노잼이었는데 ㅠㅠ 배우 연기는 좋았다,0,0,0,2,0,0.000,-0.769,-0.769,neutral
3,대박 갓작!! 재관람 의사 100% 🤩🤩,대박 갓작! 재관람 의사 100% EMOJI EMOJI,1,0,0,0,0,0.989,0.000,1.593,positive
4,좌석별로 시야가 너무 별로였음 😞,좌석별로 시야가 너무 별로였음 EMOJI,-1,0,0,0,0,-0.485,0.000,-1.291,negative
5,그럭저럭이었지만 ㅎㅎ 분위기는 좋았음,그럭저럭이었지만 ㅎㅎ 분위기는 좋았음,0,0,2,0,0,0.000,0.549,0.549,neutral
6,콘서트 진행엉망.. 최악 😭😭,콘서트 진행엉망.. 최악 EMOJI EMOJI,-1,0,0,0,0,-1.099,0.000,-1.659,negative
